In [1]:
import numpy as np
import pandas as pd
import tqdm
from scipy import signal
from pathlib import Path

from dataset_processor import (
    AddGravityColumn,
    Convert_G_to_Ms2,
    ButterworthFilter,
    Interpolate,
    Resampler,
    ResamplerPoly,
    Windowize,
    AddStandardActivityCode,
    RenameColumns,
    SplitGuaranteeingAllClassesPerSplit,
    BalanceToMinimumClass,
    Pipeline
)

In [2]:
def read_uci(uci_path):
    """Le o dataset do motionsense e retorna um dataframe com os dados (vindos de todos os arquivos CSV)
    O dataframe retornado possui as seguintes colunas:
    - attitude.roll: Rotação em torno do eixo x
    - attitude.pitch: Rotação em torno do eixo y
    - attitude.yaw: Rotação em torno do eixo z
    - gravity.x: Gravidade em torno do eixo x
    - gravity.y: Gravidade em torno do eixo y
    - gravity.z: Gravidade em torno do eixo z
    - rotationRate.x: Velocidade angular em torno do eixo x
    - rotationRate.y: Velocidade angular em torno do eixo y
    - rotationRate.z: Velocidade angular em torno do eixo z
    - userAcceleration.x: Aceleração no eixo x
    - userAcceleration.y: Aceleração no eixo y
    - userAcceleration.z: Aceleração no eixo z
    - activity code: Código da atividade
    - index: Índice da amostra vindo do txt
    - user: Usuário que realizou a atividade
    - serial: Número de série da atividade
    - txt: Caminho do txt que contém a atividade

    Parameters
    ----------
    uci_path : str
        Caminho para o dataset MotionSense

    Returns
    -------
    pd.DataFrame
        Dataframe com os dados do dataset UCI-HAR
    """
    activity_names = {
        1: "WALKING", 
        2: "WALKING_UPSTAIRS", 
        3: "WALKING_DOWNSTAIRS", 
        4: "SITTING", 
        5: "STANDING", 
        6: "LAYING",
        7: "STAND_TO_SIT",
        8: "SIT_TO_STAND",
        9: "SIT_TO_LIE",
        10: "LIE_TO_SIT",
        11: "STAND_TO_LIE",
        12: "LIE_TO_STAND"
    }
    activity_codes = {v: k for k, v in activity_names.items()}
    
    feature_columns = [
        "accel-x",
        "accel-y",
        "accel-z",
        "gyro-x",
        "gyro-y",
        "gyro-z",
    ]
    
#     df_labels = pd.read_csv("data/RawData/labels.txt", header=None, sep=" ")
    df_labels = pd.read_csv(uci_path / "labels.txt", header=None, sep=" ")
    df_labels.columns=["serial", "user", "activity code", "start", "end"]
    
    uci_path = Path(uci_path)
    
    dfs = []
    data_path = list(uci_path.glob("*.txt"))
    new_data_path = [elem.name.split("_")+[elem] for elem in sorted(data_path)]
    df = pd.DataFrame(new_data_path, columns=["sensor", "serial", "user", "file"])
    for key, df2 in df.groupby(["serial", "user"]):
        acc, gyr = None, None
        for row_index, row in df2.iterrows():
            data = pd.read_csv(row["file"], header=None, sep=" ")
            if row["sensor"] == "acc":
                acc = data
            else:
                gyr = data
        new_df = pd.concat([acc, gyr], axis=1)
        new_df.columns = feature_columns
        
        user = int(key[1].split(".")[0][4:])
        serial = int(key[0][3:])
        
        new_df['txt'] = row["file"]
        
        new_df["user"] = user
        new_df["serial"] = serial
#         new_df["activity code"] = -1
        
        for row_index, row in df_labels.loc[(df_labels["serial"] == serial) & (df_labels["user"] == user)].iterrows():
            start = row['start']
            end = row["end"]+1
            activity = row["activity code"]
            resumed_df = new_df.loc[start:end].copy()
            resumed_df["index"] = [i for i in range(start, end+1)]
            resumed_df["activity code"] = activity
            
            dfs.append(resumed_df)
    
    df = pd.concat(dfs)
    df.reset_index(inplace=True, drop=True)
    return df

In [3]:
# Caminho para o dataset UCI-HAR
uci_path = Path("../data/raw/UCI/RawData/")
# Caminho para salvar o dataset pré-processado
output_path = Path("data/processed/UCI")
# Cria o caminho de saída se ele não existir
output_path.mkdir(parents=True, exist_ok=True)

# Lista com as colunas que são features
feature_columns = [
    "accel-x",
    "accel-y",
    "accel-z",
    "gyro-x",
    "gyro-y",
    "gyro-z",
]

# Nome das colunas que serão usada para agrupar as janelas
column_group = ["user", "activity code", "serial"]

# activity code: standard activity code
standard_activity_code_map = {
    1: 2, # walk
    2: 3, # stair up
    3: 4, # stair down
    4: 0, # sit
    5: 1, # stand
    6: -1, # Laying
    7: -1, # stand to sit
    8: -1, # sit to stand
    9: -1, # sit to lie
    10: -1, # lie to sit
    11: -1, # stand to lie
    12: -1 # lie to stand
}

## Bruto

In [4]:
# Lê o dataset
dataframe = read_uci(uci_path)


# Instancia o objeto que cria as janelas
windowizer = Windowize(
    features_to_select=feature_columns,     # Nome das colunas que serão usadas como features
    samples_per_window=150,                 # Numero de amostras por janela
    samples_per_overlap=0,                  # Numero de amostras que se sobrepõem
    groupby_column=column_group,            # Agrupa pela coluna do txt. As janelas são criadas para cada grupo da coluna txt
)

# Instancia o objeto que adiciona a coluna com o código da atividade
standard_label_adder = AddStandardActivityCode(standard_activity_code_map)

# Cria o pipeline
# 1. Cria as janelas
# 2. Adiciona a coluna com o código da atividade
pipeline = Pipeline(
    [
        windowizer,
        standard_label_adder
    ]
)

# Executa o pipeline
new_df = pipeline(dataframe)
# Salva os dados
new_df.to_csv(output_path / "raw_unbalanced.csv", index=False)
new_df

Executing Windowize


Creating windows: 100%|██████████| 714/714 [00:19<00:00, 36.23it/s]


Executing AddStandardActivityCode


accel-x-0 accel-x-1 accel-x-2 accel-x-3 accel-x-4 accel-x-5 accel-x-6  \
0     1.002778  0.683333  0.733333  0.956944      1.05  1.013889      0.95   
1     1.323611  1.470833  1.470833  1.270833  1.015278  1.015278  0.888889   
2     0.890278  0.858333  0.858333  0.840278     0.925  1.086111  1.222222   
3     0.686111  0.719444  1.097222  1.097222  1.011111  0.848611  0.848611   
4     0.851389  0.897222  0.943056  0.938889  0.894445  0.868056  0.880556   
...        ...       ...       ...       ...       ...       ...       ...   
5108  0.045833  0.047222  0.072222       0.1  0.134722  0.191667  0.206944   
5109  0.073611  0.072222     0.075    0.0875  0.106944  0.126389  0.122222   
5110  1.065278  1.036111  0.916667  0.809722  0.816667  0.933333  1.090278   
5111  0.997222  1.005556  0.988889  0.988889  1.005556  0.976389  0.961111   
5112  0.206944       0.2  0.180556  0.166667  0.165278  0.163889  0.166667   

     accel-x-7 accel-x-8 accel-x-9  ... gyro-z-146 gyro-z-147 gyro-z-148  \
0         0.95  0.952778  0.913889  ...  -0.559247  -0.533285  -0.651488   
1     0.694444  0.694444  0.676389  ...   0.163406   0.180816   0.135001   
2     1.223611  1.193056  1.208333  ...   0.718683    0.93157   0.271224   
3     0.808333  0.754167  0.759722  ...  -0.161879  -0.171042  -0.127671   
4       0.9375  0.986111  0.981945  ...   0.377209   0.076053  -0.118202   
...        ...       ...       ...  ...        ...        ...        ...   
5108  0.115278  0.098611  0.054167  ...   -0.59987  -0.612392  -0.596815   
5109  0.122222  0.076389  0.009722  ...  -0.258701   -0.26481  -0.280998   
5110  1.161111  1.172222  1.090278  ...   0.028711   0.046426  -0.022297   
5111  0.968056  1.041667  1.177778  ...  -0.463036  -0.535423   -0.37782   
5112  0.158333  0.151389  0.148611  ...   0.112094   0.186008   0.247095   

     gyro-z-149  user                                            txt  index  \
0     -0.625526   1.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7496   
1      0.154549   1.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7646   
2     -0.105069   1.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7796   
3     -0.087965   1.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7946   
4     -0.180511   1.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   8372   
...         ...   ...                                            ...    ...   
5108  -0.619417  30.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   8558   
5109  -0.313374  30.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   5658   
5110  -0.024435  30.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   7132   
5111  -0.336281  30.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   4198   
5112   0.240986  30.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt  11762   

     activity code serial standard activity code  
0                1      1                      2  
1                1      1                      2  
2                1      1                      2  
3                1      1                      2  
4                1      1                      2  
...            ...    ...                    ...  
5108            10     60                     -1  
5109            10     61                     -1  
5110            11     60                     -1  
5111            11     61                     -1  
5112            12     60                     -1  

[5113 rows x 906 columns]

In [5]:
# Balanceia os dados (exlcuindo as linhas com atividade -1)
train_df, test_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user",
    class_column="standard activity code",
    train_size=0.8,
    random_state=42
)(new_df[new_df["standard activity code"] != -1])

train_df, val_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user",
    class_column="standard activity code",
    train_size=0.9,
    random_state=42
)(train_df)

balancer = BalanceToMinimumClass(class_column="standard activity code")
train_df = balancer(train_df)
val_df = balancer(val_df)
test_df = balancer(test_df)

ouptut_dir = output_path / "raw_balanced"
ouptut_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(ouptut_dir / "train.csv", index=False)
val_df.to_csv(ouptut_dir / "validation.csv", index=False)
test_df.to_csv(ouptut_dir / "test.csv", index=False)


## Normatizado com reamostrador ResamplerPoly

In [6]:
# Lê o dataset
dataframe = read_uci(uci_path)

# Instancia o objeto que converte a aceleração para m/s²
conversor = Convert_G_to_Ms2(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro que serão convertidas
)

# Instancia o objeto que aplica o filtro Butterworth
butterworth = ButterworthFilter(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro em que o filtro será aplicado
    fs=50                                                       # Frequência de amostragem original
)

# Instacia o objeto que reamostra os dados para 20Hz (supondo que o dataset original é 50Hz, constante)
resampler = ResamplerPoly(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    up=2,                                                       # Frequência de amostragem original
    down=5,                                                     # Frequência de amostragem desejada
    groupby_column=column_group,                                # Agrupa pela coluna do txt. A reamostragem é feita para cada grupo da coluna txt
)

# Instancia o objeto que cria as janelas
windowizer = Windowize(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    samples_per_window=60,                                      # Numero de amostras por janela 
    samples_per_overlap=0,                                      # Numero de amostras que se sobrepõem
    groupby_column=column_group,                                # Agrupa pela coluna do txt. As janelas são criadas para cada grupo da coluna CSV
)

# Instancia o objeto que adiciona a coluna com o código da atividade
standard_label_adder = AddStandardActivityCode(standard_activity_code_map)

# Cria o pipeline
# 1. Converte a aceleração para m/s²
# 2. Aplica o filtro Butterworth
# 3. Reamostra os dados para 20Hz
# 4. Cria as janelas
# 5. Adiciona a coluna com o código da atividade
pipeline = Pipeline(
    [
        conversor,
        butterworth,
        resampler,
        windowizer,
        standard_label_adder
    ]
)

# Executa o pipeline
new_df_standartized = pipeline(dataframe)
# Salva os dados
new_df_standartized.to_csv(output_path / "standartized_unbalanced.csv", index=False)
new_df_standartized

Executing Convert_G_to_Ms2
Executing ButterworthFilter
Executing ResamplerPoly


Resampling: 100%|██████████| 714/714 [00:02<00:00, 281.38it/s]


Executing Windowize


Creating windows: 100%|██████████| 714/714 [00:20<00:00, 34.73it/s]


Executing AddStandardActivityCode


accel-x-0 accel-x-1 accel-x-2 accel-x-3 accel-x-4 accel-x-5 accel-x-6  \
0    -1.294861 -1.821932  0.177586 -1.053442 -0.923933 -0.594807 -0.876665   
1     2.534239  3.831228 -0.723642 -2.601742 -3.328371 -2.629107 -2.048427   
2    -0.559527 -1.948379   1.03147  1.863631  2.848757 -0.354659  1.650163   
3    -3.345663  0.554403      -0.9 -2.485545 -1.182032 -0.612004 -0.443396   
4    -1.496035 -0.508597 -1.306504 -0.350937 -0.499819  0.159376  1.249205   
...        ...       ...       ...       ...       ...       ...       ...   
5115  0.150832  0.260999  0.655816  0.227044 -0.573703 -0.059493 -0.229176   
5116  0.503664 -1.466862 -0.624668  1.818005  0.323614 -3.594533 -0.257988   
5117 -0.476063  -0.70046 -0.938115 -0.534469  1.455487  1.184784 -1.104806   
5118  0.377754   0.30951  0.026914  0.018463 -0.355628  0.128436  0.286268   
5119  0.322907   0.45926   0.30159   0.70434  0.538645 -0.176298 -0.489825   

     accel-x-7 accel-x-8 accel-x-9  ... gyro-z-57 gyro-z-58 gyro-z-59  user  \
0     0.012722  0.072624  -0.04399  ... -0.482817 -0.415797 -0.611074   1.0   
1     1.414561  2.372414   3.57132  ... -0.288663 -0.037839  0.181263   1.0   
2     4.927537 -1.608229 -2.495104  ... -0.177897  0.323679  0.559186   1.0   
3     0.193144  0.227264  1.523604  ...  0.220599   0.00123  -0.17577   1.0   
4     2.242568   1.14331 -0.926829  ...  0.121477   0.45223 -0.029443   1.0   
...        ...       ...       ...  ...       ...       ...       ...   ...   
5115 -0.158066  -0.55108 -0.628638  ... -0.158095  -0.27472 -0.261726  30.0   
5116  2.925112  0.876846  0.648942  ...  0.004621  0.068806 -0.001791  30.0   
5117 -1.751874 -1.944943 -0.715098  ... -0.621379 -0.423297 -0.476595  30.0   
5118  0.240792  0.081811  0.076491  ...  0.026466  0.083232  0.203615  30.0   
5119  0.442305  1.164235  0.329035  ... -0.385935 -0.310453  0.252967  30.0   

       level_0                                            txt  index  \
0       6740.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7496   
1       6800.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7556   
2       6860.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7616   
3       6920.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7676   
4       6980.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7736   
...        ...                                            ...    ...   
5115  807372.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   5658   
5116  790101.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   7132   
5117  805910.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   4198   
5118  794737.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt  11762   
5119  810243.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   8525   

     activity code serial standard activity code  
0                1      1                      2  
1                1      1                      2  
2                1      1                      2  
3                1      1                      2  
4                1      1                      2  
...            ...    ...                    ...  
5115            10     61                     -1  
5116            11     60                     -1  
5117            11     61                     -1  
5118            12     60                     -1  
5119            12     61                     -1  

[5120 rows x 367 columns]

In [7]:
# Balanceia os dados (exlcuindo as linhas com atividade -1)
train_df, test_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.8,
    random_state=42
)(new_df_standartized[new_df_standartized["standard activity code"] != -1])

train_df, val_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.9,
    random_state=42
)(train_df)

balancer = BalanceToMinimumClass(class_column="standard activity code")
train_df = balancer(train_df)
val_df = balancer(val_df)
test_df = balancer(test_df)

ouptut_dir = output_path / "standartized_balanced" 
ouptut_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(ouptut_dir / "train.csv", index=False)
val_df.to_csv(ouptut_dir / "validation.csv", index=False)
test_df.to_csv(ouptut_dir / "test.csv", index=False)

## Normatizado com reamostrador Resampler

In [8]:
# Lê o dataset
dataframe = read_uci(uci_path)

# Instancia o objeto que converte a aceleração para m/s²
conversor = Convert_G_to_Ms2(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro que serão convertidas
)

# Instancia o objeto que aplica o filtro Butterworth
butterworth = ButterworthFilter(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro em que o filtro será aplicado
    fs=50                                                       # Frequência de amostragem original
)

# Instacia o objeto que reamostra os dados para 20Hz (supondo que o dataset original é 50Hz, constante)
resampler = Resampler(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    original_fs=50,                                             # Frequência de amostragem original
    target_fs=20,                                               # Frequência de amostragem desejada
    groupby_column=column_group,                                # Agrupa pela coluna do CSV. A reamostragem é feita para cada grupo da coluna CSV
)

# Instancia o objeto que cria as janelas
windowizer = Windowize(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    samples_per_window=60,                                      # Numero de amostras por janela 
    samples_per_overlap=0,                                      # Numero de amostras que se sobrepõem
    groupby_column=column_group,                                # Agrupa pela coluna do txt. As janelas são criadas para cada grupo da coluna CSV
)

# Instancia o objeto que adiciona a coluna com o código da atividade
standard_label_adder = AddStandardActivityCode(standard_activity_code_map)

# Cria o pipeline
# 1. Converte a aceleração para m/s²
# 2. Aplica o filtro Butterworth
# 3. Reamostra os dados para 20Hz
# 4. Cria as janelas
# 5. Adiciona a coluna com o código da atividade
pipeline = Pipeline(
    [
        conversor,
        butterworth,
        resampler,
        windowizer,
        standard_label_adder
    ]
)

# Executa o pipeline
new_df_standartized = pipeline(dataframe)
# Salva os dados
new_df_standartized.to_csv(output_path / "standartized_unbalanced_2.csv", index=False)
new_df_standartized

Executing Convert_G_to_Ms2
Executing ButterworthFilter
Executing Resampler


Resampling: 100%|██████████| 714/714 [00:01<00:00, 359.60it/s]


Executing Windowize


Creating windows: 100%|██████████| 714/714 [00:20<00:00, 35.07it/s]


Executing AddStandardActivityCode


accel-x-0 accel-x-1 accel-x-2 accel-x-3 accel-x-4 accel-x-5 accel-x-6  \
0    -2.833257 -1.206962 -0.222144 -0.762373 -1.152283 -0.405408 -1.031523   
1     3.234248  3.319959 -1.304929 -2.615548  -3.46639 -2.351809 -1.932922   
2    -0.931924 -1.648398  1.898844  1.800302  2.746011 -1.317004  4.000048   
3    -2.149846  0.894538 -2.212124 -1.962087 -0.771756 -0.632305  -0.10154   
4    -0.789999 -0.953819 -0.928713 -0.201753 -0.429709  0.960063  1.756872   
...        ...       ...       ...       ...       ...       ...       ...   
5108 -0.436421 -0.123993  0.672125 -0.617835 -1.199618 -2.236434 -3.342621   
5109  0.100062  0.382779  0.592919 -0.319994 -0.281877 -0.082347 -0.320796   
5110   0.54159 -1.556119 -0.346829  1.927838 -0.515841 -3.571364  1.282897   
5111 -0.671581 -0.533349 -1.103813 -0.367796  1.291531  1.338266 -1.245868   
5112  0.379818    0.2795  0.026941 -0.045978 -0.332621  0.306278  0.203183   

     accel-x-7 accel-x-8 accel-x-9  ... gyro-z-57 gyro-z-58 gyro-z-59  user  \
0     0.163501 -0.054556  0.063171  ... -0.496033 -0.425188 -0.630119   1.0   
1     2.151541  2.061283  4.162129  ...  -0.32155  0.090746  0.155261   1.0   
2     2.971553 -2.321007 -2.870096  ... -0.082779  0.609134  0.255944   1.0   
3     0.199842  0.575738  2.370602  ...  0.185679 -0.182803 -0.080921   1.0   
4      2.14327 -0.603573   0.44661  ...  0.427588  0.131001 -0.144445   1.0   
...        ...       ...       ...  ...       ...       ...       ...   ...   
5108 -2.214512 -1.812965 -1.319833  ... -0.137071  0.054419  0.138349  30.0   
5109 -0.565574 -0.481863   0.46418  ...  -0.19606 -0.065447 -0.006044  30.0   
5110  2.458398  0.428781    1.0902  ... -0.025263  0.049848  0.010395  30.0   
5111 -1.630863 -2.034239 -0.659126  ... -0.633716 -0.410474 -0.488779  30.0   
5112  0.247296 -0.021078  0.022257  ...  0.232847   0.07677  0.023796  30.0   

       level_0                                            txt  index  \
0       6740.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7496   
1       6800.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7556   
2       6860.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7616   
3       6920.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7676   
4       6980.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7736   
...        ...                                            ...    ...   
5108  791529.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   8558   
5109  807372.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   5658   
5110  790101.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   7132   
5111  805910.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   4198   
5112  794737.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt  11762   

     activity code serial standard activity code  
0                1      1                      2  
1                1      1                      2  
2                1      1                      2  
3                1      1                      2  
4                1      1                      2  
...            ...    ...                    ...  
5108            10     60                     -1  
5109            10     61                     -1  
5110            11     60                     -1  
5111            11     61                     -1  
5112            12     60                     -1  

[5113 rows x 367 columns]

In [9]:
# Balanceia os dados (exlcuindo as linhas com atividade -1)
train_df, test_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.8,
    random_state=42
)(new_df_standartized[new_df_standartized["standard activity code"] != -1])

train_df, val_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.9,
    random_state=42
)(train_df)

balancer = BalanceToMinimumClass(class_column="standard activity code")
train_df = balancer(train_df)
val_df = balancer(val_df)
test_df = balancer(test_df)

ouptut_dir = output_path / "standartized_balanced_2" 
ouptut_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(ouptut_dir / "train.csv", index=False)
val_df.to_csv(ouptut_dir / "validation.csv", index=False)
test_df.to_csv(ouptut_dir / "test.csv", index=False)

## Normatizado com Interpolador

In [10]:
# Lê o dataset
dataframe = read_uci(uci_path)

# Instancia o objeto que converte a aceleração para m/s²
conversor = Convert_G_to_Ms2(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro que serão convertidas
)

# Instancia o objeto que aplica o filtro Butterworth
butterworth = ButterworthFilter(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro em que o filtro será aplicado
    fs=50                                                       # Frequência de amostragem original
)

# Instacia o objeto que interpola os dados para 20Hz (supondo que o dataset original é 50Hz, constante)
interpolator = Interpolate(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    original_fs=50,                                             # Frequência de amostragem original (50Hz)
    target_fs=20,                                               # Frequência de amostragem desejada (20Hz)
    kind="cubic",                                               # Tipo de interpolação (cúbica)
    groupby_column=column_group,                                # Agrupa pela coluna do CSV. A reamostragem é feita para cada grupo da coluna CSV
)

# Instancia o objeto que cria as janelas
windowizer = Windowize(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    samples_per_window=60,                                      # Numero de amostras por janela 
    samples_per_overlap=0,                                      # Numero de amostras que se sobrepõem
    groupby_column=column_group,                                # Agrupa pela coluna do txt. As janelas são criadas para cada grupo da coluna CSV
)

# Instancia o objeto que adiciona a coluna com o código da atividade
standard_label_adder = AddStandardActivityCode(standard_activity_code_map)

# Cria o pipeline
# 1. Converte a aceleração para m/s²
# 2. Aplica o filtro Butterworth
# 3. Reamostra os dados para 20Hz
# 4. Cria as janelas
# 5. Adiciona a coluna com o código da atividade
pipeline = Pipeline(
    [
        conversor,
        butterworth,
        interpolator,
        windowizer,
        standard_label_adder
    ]
)

# Executa o pipeline
new_df_standartized = pipeline(dataframe)
# Salva os dados
new_df_standartized.to_csv(output_path / "standartized_unbalanced_3.csv", index=False)
new_df_standartized

Executing Convert_G_to_Ms2
Executing ButterworthFilter
Executing Interpolate


Interpoling: 100%|██████████| 714/714 [00:03<00:00, 201.39it/s]


Executing Windowize


Creating windows: 100%|██████████| 714/714 [00:19<00:00, 36.97it/s]


Executing AddStandardActivityCode


accel-x-0 accel-x-1 accel-x-2 accel-x-3 accel-x-4 accel-x-5 accel-x-6  \
0    -0.108253  -1.70682 -0.092498 -0.692162 -1.152006 -0.523794 -1.002162   
1     3.139491  3.842262  0.103704 -3.178486 -2.930759 -3.150714  -1.55078   
2    -1.113408 -1.565752  0.804432    1.9822  2.803848  -0.39764  1.793148   
3    -3.139514  1.304302 -1.544931  -2.23622 -1.285103 -0.495753 -0.330518   
4    -1.444715 -0.497251 -1.285537 -0.323746 -0.623222  0.369419  1.046437   
...        ...       ...       ...       ...       ...       ...       ...   
5109 -0.494031 -0.192878  0.739325 -0.262871 -1.301882 -1.272691 -3.089791   
5110  0.242739  0.256107  0.649913  0.389043 -0.516501  0.073267 -0.317146   
5111  0.499667  -1.50002 -0.640161  1.849481  0.479428 -3.452753  0.714971   
5112 -0.673962 -0.807038 -0.900033 -0.742999  1.438109    1.0699 -0.860282   
5113  0.597614  0.211283  0.080506 -0.074051 -0.250727  0.092202  0.336657   

     accel-x-7 accel-x-8 accel-x-9  ... gyro-z-57 gyro-z-58 gyro-z-59  user  \
0     0.279494 -0.000654 -0.095793  ... -0.555096 -0.365297 -0.586242   1.0   
1     0.693905  3.235847  2.689402  ... -0.205649 -0.022297  0.156366   1.0   
2     5.239036 -1.342308 -2.738454  ... -0.042361  0.096517  0.648466   1.0   
3     0.316147   0.18847  1.477778  ...  0.244827 -0.012217 -0.151539   1.0   
4     2.400931  1.108601 -0.578266  ...  0.127686  0.439212 -0.041975   1.0   
...        ...       ...       ...  ...       ...       ...       ...   ...   
5109 -3.006334 -2.294193 -1.614869  ... -0.516208 -0.569632 -0.603173  30.0   
5110 -0.129901 -0.538235 -0.605045  ... -0.158479 -0.272751  -0.27274  30.0   
5111  2.236111  0.940236  0.155642  ...  0.037555  0.050091  0.015477  30.0   
5112  -1.94903 -1.620573  -1.37326  ... -0.646051  -0.35644 -0.463219  30.0   
5113  0.117289   0.09708  0.144579  ...  0.037119  0.076664   0.22264  30.0   

       level_0                                            txt  index  \
0       6740.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7496   
1       6800.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7556   
2       6860.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7616   
3       6920.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7676   
4       6980.0  ../data/raw/UCI/RawData/gyro_exp01_user01.txt   7736   
...        ...                                            ...    ...   
5109  791529.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   8558   
5110  807372.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   5658   
5111  790101.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt   7132   
5112  805910.0  ../data/raw/UCI/RawData/gyro_exp61_user30.txt   4198   
5113  794737.0  ../data/raw/UCI/RawData/gyro_exp60_user30.txt  11762   

     activity code serial standard activity code  
0                1      1                      2  
1                1      1                      2  
2                1      1                      2  
3                1      1                      2  
4                1      1                      2  
...            ...    ...                    ...  
5109            10     60                     -1  
5110            10     61                     -1  
5111            11     60                     -1  
5112            11     61                     -1  
5113            12     60                     -1  

[5114 rows x 367 columns]

In [11]:
# Balanceia os dados (exlcuindo as linhas com atividade -1)
train_df, test_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.8,
    random_state=42
)(new_df_standartized[new_df_standartized["standard activity code"] != -1])

train_df, val_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.9,
    random_state=42
)(train_df)

balancer = BalanceToMinimumClass(class_column="standard activity code")
train_df = balancer(train_df)
val_df = balancer(val_df)
test_df = balancer(test_df)

ouptut_dir = output_path / "standartized_balanced_3" 
ouptut_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(ouptut_dir / "train.csv", index=False)
val_df.to_csv(ouptut_dir / "validation.csv", index=False)
test_df.to_csv(ouptut_dir / "test.csv", index=False)